In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Öncelikle verimizi ekliyoruz.

In [ ]:
data=pd.read_csv('../input/diabetes.csv')
data.head()

In [ ]:
data.info()

Verimize ait karar sınıfı olan Outcome sınıfındaki verilerin dağılımına bakıyoruz. Görüldüğü gibi '1' sınıfından 268 '0' sınıfından ise 500 adet veri bulunmakta.

In [ ]:
data.Outcome.value_counts()

Şimdi ise karar sınıfımız ve bu karar sınıfını etkileten özelliklerimizi ayırmaya yani x ve y veri setlerimizi hazırlama aşamasındayız. y bizim karar sınıfımız, x ise karar sınıfı dışında bulunan bütün veri setini temsil etmekte.

In [ ]:
y=data.Outcome.values

In [ ]:
x_data=data.iloc[:,:8]
x_data.head()

işlemleri yaparken büyük veri değerlerimizin küçük veri değerlerimizden daha fazla baskın olmaması için normalizasyon çalışması yapılmakta. bütün y veri setimizin tamamına normalizasyon işlemi uyguluyoruz.

In [ ]:
x=(x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))
x.head()

                Veri setimizin belirli bir bölümünü eğitim diğer bölümünü de test veri seti olarak ayırma işlemini sklearn kütüphanesini kullanarak gerçekleştiriyoruz. Burada train_test_split ile verimizin %80'ini eğitim, %20'sini ise test seti olarak kullanacağız ve her seferinde bu veri dağılımı seçilirken rasgele değerler seçiminin önüne geçmek için random_state değerine 42 değerini verdik.
                Daha sonra ise verimizi bir YSA örneği gibi modelimize vereceğimiz için her özelliğe ait değeri verebilmek için matrisin Transpozunu alacağız. 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("x_train: ",x_train.shape)
print("x_test: ",x_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

    Aşağıdaki fonksiyon yardımıya weight ve bias değerlerinin başlangıç değerini belirliyoruz. Burada modelimize gelecek olan x girişi sayısını dimension adlı parametre ile almaktayız.

In [ ]:
def initialize_weights_and_bias(dimension):
    
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b



sigmoid fonksiyonu ile w*x+b işlemleri sonucu gelen değerin belirli bir aralıkta olmasını sağlayan işlemimizi gerçekleştiriyoruz.

In [ ]:
def sigmoid(z):
    
    y_head = 1/(1+ np.exp(-z))
    return y_head





forward_backward_propagation fonksiyonu ile, forward ve backward propagation işlemlerini gerçekleştiriyoruz. Bu fonksiyonu ağırlıkları ve bias değerlerini güncelleme aşamasında kullanacağız. w ve b değerlerinin optimum değerlerine ulaşana kadar sırasıyla ileri-geri yayılım işlemini gerçekleştireceğiz.

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]      
    
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost,gradients




Şimdi sıra geldi w ve b değerlerini güncellemeye. Burada belirlenen iterasyona göre w ve b değerleri güncellenecek ve forward_backward_propagation fonksiyonu kullanılarak loss ve cost değerlerinin minimumuna ulaşılmaya çalışılacaktır.

In [ ]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
            
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

Şimdiye kadar yaptığımız işlem modelimizi oluşturmak için eğitme aşamasıydı. Bundan sonra ise oluşturduğumuz modelin ne kadar verimli bir model olduğunu test aşamasına sıra geldi. Bunun için daha önce ayırdığımız test veri setimizi kullanarak modelimizin daha önce görmediği veride nasıl sonuç üreteceğini test edeceğiz.

In [ ]:
def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    # if z is bigger than 0.5, our prediction is sign one (y_head=1),
    # if z is smaller than 0.5, our prediction is sign zero (y_head=0),
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):
    # initialize
    dimension =  x_train.shape[0]  # that is 30
    w,b = initialize_weights_and_bias(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)

    # Print test Errors
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))

modelimizi uyguladığımızda hangi iterasyonda nasıl sonuç aldığımızı aşağıda görmekteyiz. Görüldüğü gibi belirli bir iterasyondan sonra aynı sonuçları aldığımız görülmektedir. 

In [ ]:
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 1, num_iterations = 600)  


In [ ]:
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 1, num_iterations = 700) 

In [ ]:
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 1, num_iterations = 800) 

In [ ]:
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 1, num_iterations = 830) 

Yukarıdaki aşamalarda tamamen işlemleri normal matematik kullanarak gerçekleştirdik. Fakat sklearn kütüphanesinin linear regression classification işlemi için kullanabileceğimiz bir kütüphanesi mevcut. Bu işlem için verimizi eğitim ve test veri seti olarak böldükten sonra doğrudan işlemi gerçekleştirebiliriz. Görüldüğü üzere bizim yukarıda bulduğumuz sonuçlara benzer değer üretilmiştir.

In [ ]:
y1=data.Outcome.values
x1d=data.iloc[:,:8]
x1=(x1d-np.min(x1d))/(np.max(x1d)-np.min(x1d))
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.2,random_state=42)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train1,y_train1)
print(lr.get_params())
print("test accuracy {}".format(lr.score(x_test1,y_test1)))
